In [1]:
import tensorflow as tf
from scipy.io import loadmat
import scipy.sparse as sparse
import numpy as np
from datetime import datetime


# Encoding in .tfRecords from .Mat file

In [3]:
train_addrs = ["MatlabData/trial_"+str(i+1)+".mat" for i in range(7361) ]
train_addrs[0:5]

size_batch_in_memory = 1000
num_batches = len(train_addrs)//size_batch_in_memory +((len(train_addrs)%size_batch_in_memory)>0)

#Addresses to save the TFRecords file
train_filename = ["train_"+str(i+1)+".tfrecords" for i in range(num_batches)]
train_filename



['train_1.tfrecords',
 'train_2.tfrecords',
 'train_3.tfrecords',
 'train_4.tfrecords',
 'train_5.tfrecords',
 'train_6.tfrecords',
 'train_7.tfrecords',
 'train_8.tfrecords']

In [ ]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))
def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def from_sparse_matrix_to_feature(features, mapping_number,name="default"):
    sparse_array = features[mapping_number]
    index_x, index_y, values = sparse.find(sparse_array)
    shape = sparse_array.shape
    
    feature = {    name+'_x': _int64_feature(index_x), 
                   name+'_y': _int64_feature(index_y),
                   name+'_values': _float_feature(values),
                   name+'_shape':  _int64_feature(shape)
              }
    return feature 

def from_int_matrix_to_feature(features, mapping_number, name="default"):
    return {name: _int64_feature(np.array(features[mapping_number].flatten()))}

def from_float_matrix_to_feature(features, mapping_number, name="default"):
    return {name: _float_feature(np.array(features[mapping_number].flatten()))}

def from_string_matrix_to_feature(features, mapping_number, name="default"):
    return {name: _bytes_feature(np.array(features[mapping_number].flatten(), dtype='str'))}

In [ ]:
np.save('mapping_Mat_Python.npy', map_name_to_type_and_position) 


In [ ]:
map_name_to_type_and_position = np.load('mapping_Mat_Python.npy').item()

In [ ]:
#Iterate on the number of files
for j in range(num_batches):
    # open the TFRecords file
    writer = tf.python_io.TFRecordWriter(train_filename[j]) 
    
    for i in range(size_batch_in_memory):
        
        #Case where no file left
        if j*size_batch_in_memory+i>=len(train_addrs):
            writer.close()
            break
            
        # print how many examples are saved every 100 images
        if not i % 100:
            print 'Train data: {}/{}'.format(j*size_batch+i, len(train_addrs))
            sys.stdout.flush()
        
        # Load the the .mat file 
        mat = loadmat(train_addrs[j*size_batch+i])
        features = list(mat['structarr'][0,0])
        
        # Convert the features 
        feature = {}
        for k in map_name_to_type_and_position:
            map_number, map_type = map_name_to_type_and_position[k]
            if map_type == int:
                feature.update(from_int_matrix_to_feature(features, map_number, name=k))
            if map_type == float:
                feature.update(from_float_matrix_to_feature(features, map_number, name=k))
            if map_type == 'str':
                feature.update(from_string_matrix_to_feature(features, map_number, name=k))
            if map_type == 'sparse':
                feature.update(from_sparse_matrix_to_feature(features, map_number, name=k))
                

        # Create an example protocol buffer
        example = tf.train.Example(features=tf.train.Features(feature=feature))

        # Serialize to string and write on the file
        writer.write(example.SerializeToString())

    writer.close()
sys.stdout.flush()

# Decoding for training

In [14]:
tf.reset_default_graph()

In [15]:
max_len_sequence = 2568

n_steps = 200
n_steps_to_predict = 10
n_features_spikeRaster = 96
n_dims_output = 3

n_inputs = n_features_spikeRaster + n_dims_output



In [16]:
def _parse_function(example_proto):
    
    features = {
                "handPos" : tf.VarLenFeature(tf.float32),
                'spikeRaster': tf.SparseFeature(index_key=['spikeRaster_x', 'spikeRaster_y'],
                                                  value_key='spikeRaster_values',
                                                  dtype=tf.float32, size=[n_features_spikeRaster, max_len_sequence]),
                "spikeRaster_shape": tf.FixedLenFeature([2],tf.int64)               
               }
    
    parsed_features = tf.parse_single_example(example_proto, features)
    
    # Preprocess spikeRaster => [Time Series n_steps x n_features_spikeRaster]
    parsed_features["spikeRaster"] = tf.sparse_slice(parsed_features["spikeRaster"],
                                                     [0,0],parsed_features["spikeRaster_shape"])
    parsed_features["spikeRaster"] = tf.sparse_tensor_to_dense(parsed_features["spikeRaster"])
    spikeRaster = tf.reshape(tf.transpose(parsed_features["spikeRaster"]), [-1,n_features_spikeRaster])
    
    # Preprocess lengths of sequences = []
    seq_length = tf.cast(parsed_features["spikeRaster_shape"][1], tf.int32)
        
    # Preprocess handPos = [n_steps x 3] => HELPER
    handPos = tf.sparse_tensor_to_dense(parsed_features["handPos"])
    handPos = tf.transpose(tf.reshape(handPos, [n_dims_output,-1]))
    
    # Useful features
    features = tf.concat([spikeRaster,handPos], axis=1)    
    return features

def get_slices(x):
    num_slices = tf.shape(x, out_type=tf.int64)[0] - n_steps - n_steps_to_predict + 1
    return tf.data.Dataset.range(num_slices).map(lambda i: (x[i:i + n_steps] , 
                                                            x[i+n_steps:i+n_steps+n_steps_to_predict,-3:]))

In [17]:
num_epochs = 5
size_batch = 32
filenames = ["Data/train_"+str(i+1)+".tfrecords" for i in range(1)]

dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(_parse_function)
dataset = dataset.flat_map(get_slices)

dataset = dataset.shuffle(buffer_size=10000)
dataset = dataset.batch(size_batch)
dataset = dataset.repeat(num_epochs)

iterator = dataset.make_initializable_iterator()

inputs, real_position = iterator.get_next()

inputs.set_shape([None,n_steps,n_inputs])
real_position.set_shape([None,n_steps_to_predict,3])


In [18]:
with tf.Session() as sess:
    sess.run(iterator.initializer)
    e1, e2 = sess.run([inputs, real_position])

In [19]:
inputs, real_position

(<tf.Tensor 'IteratorGetNext:0' shape=(?, 200, 99) dtype=float32>,
 <tf.Tensor 'IteratorGetNext:1' shape=(?, 10, 3) dtype=float32>)

In [20]:
is_training = tf.placeholder(tf.bool, [])

keep_prob = 0.5
num_units = 128

# Build RNN cell
encoder_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units)

# Run Dynamic RNN
# inputs: [batch_size, n_steps, n_inputs]
rnn_outputs, _ = tf.nn.dynamic_rnn(
    encoder_cell, inputs,
    time_major=False,
    dtype = tf.float32)

# Recover meaningful outputs // Predict 3D positions
rnn_outputs = rnn_outputs[:,- n_steps_to_predict:,:]
predicted_position = tf.layers.dense(rnn_outputs, 3)

loss = tf.losses.mean_squared_error(predictions=predicted_position, labels=real_position)
training_op = tf.train.AdamOptimizer().minimize(loss)

In [21]:
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

mse_summary = tf.summary.scalar('MSE', loss)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [22]:
saver = tf.train.Saver()
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        sess.run(iterator.initializer)
        current_loss = sess.run(loss)
        n_iteration = 0
        print("Loss after {} epochs: {}".format(epoch,current_loss))
        
        while True:
            try:
                _, current_loss = sess.run([training_op,loss])
                n_iteration += 1
                
                if not(n_iteration%100):
                    print("Loss after {} iteration: {}".format(n_iteration,current_loss)) 
            except tf.errors.OutOfRangeError:
                saver.save(sess, "/ModelWeights/test_model.ckpt")

Loss after 0 epochs: 68937.8828125
Loss after 100 iteration: 1532.94726562
Loss after 200 iteration: 17520.5253906
Loss after 300 iteration: 49794.5820312
Loss after 400 iteration: 17232.7265625
Loss after 500 iteration: 16982.90625
Loss after 600 iteration: 585.482971191
Loss after 700 iteration: 16359.5449219
Loss after 800 iteration: 228.598861694
Loss after 900 iteration: 175.397598267
Loss after 1000 iteration: 186.21282959
Loss after 1100 iteration: 15859.78125
Loss after 1200 iteration: 153.671188354
Loss after 1300 iteration: 149.281417847
Loss after 1400 iteration: 89.3643722534
Loss after 1500 iteration: 53.9564590454
Loss after 1600 iteration: 89.3916320801
Loss after 1700 iteration: 72.5566253662
Loss after 1800 iteration: 109.30480957
Loss after 1900 iteration: 96.0612411499
Loss after 2000 iteration: 58.0750999451
Loss after 2100 iteration: 60.0071792603
Loss after 2200 iteration: 56.8306236267
Loss after 2300 iteration: 15.4345197678
Loss after 2400 iteration: 52.4061622

Loss after 19900 iteration: 60695.5039062
Loss after 20000 iteration: 41887.890625
Loss after 20100 iteration: 41301.390625
Loss after 20200 iteration: 40937.25
Loss after 20300 iteration: 11684.4121094
Loss after 20400 iteration: 5784.11865234
Loss after 20500 iteration: 1.7737621069
Loss after 20600 iteration: 1.34397566319
Loss after 20700 iteration: 5740.24902344
Loss after 20800 iteration: 1.99288570881
Loss after 20900 iteration: 1.47819638252
Loss after 21000 iteration: 0.618279159069
Loss after 21100 iteration: 72757.6171875
Loss after 21200 iteration: 60031.2734375
Loss after 21300 iteration: 31606.4023438
Loss after 21400 iteration: 31332.1308594
Loss after 21500 iteration: 27852.1542969
Loss after 21600 iteration: 15930.3105469
Loss after 21700 iteration: 15636.6962891
Loss after 21800 iteration: 20673.4199219
Loss after 21900 iteration: 5074.95947266
Loss after 22000 iteration: 38268.859375
Loss after 22100 iteration: 57542.5
Loss after 22200 iteration: 37222.8632812
Loss a

Loss after 39400 iteration: 0.696436285973
Loss after 39500 iteration: 1.85825920105
Loss after 39600 iteration: 1.90996527672
Loss after 39700 iteration: 1.71395981312
Loss after 39800 iteration: 2.49350261688
Loss after 39900 iteration: 1.02014088631
Loss after 40000 iteration: 0.493102103472
Loss after 40100 iteration: 1.55933344364
Loss after 40200 iteration: 1.52543306351
Loss after 40300 iteration: 0.47427636385
Loss after 40400 iteration: 1.11967384815
Loss after 40500 iteration: 0.643781244755
Loss after 40600 iteration: 0.690904438496
Loss after 40700 iteration: 0.89929163456
Loss after 40800 iteration: 0.702994942665
Loss after 40900 iteration: 0.897388696671
Loss after 41000 iteration: 1.16950154305
Loss after 41100 iteration: 0.955654263496
Loss after 41200 iteration: 1.59797108173
Loss after 41300 iteration: 0.629818320274
Loss after 41400 iteration: 1.83480978012
Loss after 41500 iteration: 0.893944501877
Loss after 41600 iteration: 1.59233236313
Loss after 41700 iteratio

Loss after 58900 iteration: 1.40390741825
Loss after 59000 iteration: 0.896083712578
Loss after 59100 iteration: 1.01911473274
Loss after 59200 iteration: 0.93811160326
Loss after 59300 iteration: 1.54628992081
Loss after 59400 iteration: 1.52103376389
Loss after 59500 iteration: 0.974042892456
Loss after 59600 iteration: 1.69408357143
Loss after 59700 iteration: 0.631433486938
Loss after 59800 iteration: 0.73981654644
Loss after 59900 iteration: 0.919718921185
Loss after 60000 iteration: 0.600532531738
Loss after 60100 iteration: 1.16440665722
Loss after 60200 iteration: 0.629459023476
Loss after 60300 iteration: 1.16586434841
Loss after 60400 iteration: 0.637820839882
Loss after 60500 iteration: 1.38982212543
Loss after 60600 iteration: 2.35700082779
Loss after 60700 iteration: 75.4774856567
Loss after 60800 iteration: 23.955450058
Loss after 60900 iteration: 74.2437210083
Loss after 61000 iteration: 7.21956014633
Loss after 61100 iteration: 10.1400957108
Loss after 61200 iteration: 

Loss after 78400 iteration: 90.6154937744
Loss after 78500 iteration: 40.6277427673
Loss after 78600 iteration: 3.02304625511
Loss after 78700 iteration: 1.74836301804
Loss after 78800 iteration: 15.4875097275
Loss after 78900 iteration: 1.72419679165
Loss after 79000 iteration: 1.818328619
Loss after 79100 iteration: 62.9749984741
Loss after 79200 iteration: 173.370803833
Loss after 79300 iteration: 22.6094760895
Loss after 79400 iteration: 11.7013368607
Loss after 79500 iteration: 3.47280526161
Loss after 79600 iteration: 5.92175483704
Loss after 79700 iteration: 2.34692645073
Loss after 79800 iteration: 21.9512958527
Loss after 79900 iteration: 1.73233950138
Loss after 80000 iteration: 15.9480857849
Loss after 80100 iteration: 2.04672837257
Loss after 80200 iteration: 0.974632561207
Loss after 80300 iteration: 2.7858235836
Loss after 80400 iteration: 1.75625991821
Loss after 80500 iteration: 1.16160941124
Loss after 80600 iteration: 1.54676103592
Loss after 80700 iteration: 0.776934

Loss after 97900 iteration: 0.984153926373
Loss after 98000 iteration: 1.04472482204
Loss after 98100 iteration: 1.832203269
Loss after 98200 iteration: 0.955803513527
Loss after 98300 iteration: 0.303664594889
Loss after 98400 iteration: 0.79790532589
Loss after 98500 iteration: 1.1100076437
Loss after 98600 iteration: 1.10985004902
Loss after 98700 iteration: 1.44940638542
Loss after 98800 iteration: 1.23051142693
Loss after 98900 iteration: 0.986417651176
Loss after 99000 iteration: 1.91396462917
Loss after 99100 iteration: 1.05740904808
Loss after 99200 iteration: 1.16314089298
Loss after 99300 iteration: 0.662886083126
Loss after 99400 iteration: 1306.22192383
Loss after 99500 iteration: 641.718200684
Loss after 99600 iteration: 6418.02197266
Loss after 99700 iteration: 128.094680786
Loss after 99800 iteration: 5.98485422134
Loss after 99900 iteration: 68.9985046387
Loss after 100000 iteration: 62.047706604
Loss after 100100 iteration: 16.5018844604
Loss after 100200 iteration: 1.

Loss after 117000 iteration: 1.2972189188
Loss after 117100 iteration: 1.21840167046
Loss after 117200 iteration: 0.892256319523
Loss after 117300 iteration: 1.48539328575
Loss after 117400 iteration: 0.982440650463
Loss after 117500 iteration: 0.438115686178
Loss after 117600 iteration: 1.42187726498
Loss after 117700 iteration: 1.39330112934
Loss after 117800 iteration: 35.1770248413
Loss after 117900 iteration: 5.81013393402
Loss after 118000 iteration: 7.36690044403
Loss after 118100 iteration: 1.79558765888
Loss after 118200 iteration: 2.09813928604
Loss after 118300 iteration: 1.44370901585
Loss after 118400 iteration: 0.716563642025
Loss after 118500 iteration: 11.708855629
Loss after 118600 iteration: 5.62614870071
Loss after 118700 iteration: 2.67799282074
Loss after 118800 iteration: 2.05282402039
Loss after 118900 iteration: 1.85617208481
Loss after 119000 iteration: 3.63927054405
Loss after 119100 iteration: 2.61839175224
Loss after 119200 iteration: 2.08677721024
Loss afte

Loss after 136100 iteration: 18.2983589172
Loss after 136200 iteration: 22.0979328156
Loss after 136300 iteration: 11147.8623047
Loss after 136400 iteration: 5.3764872551
Loss after 136500 iteration: 4343.63427734
Loss after 136600 iteration: 5.32090044022
Loss after 136700 iteration: 3.70740509033
Loss after 136800 iteration: 6.37613630295
Loss after 136900 iteration: 2.66247320175
Loss after 137000 iteration: 0.851670444012
Loss after 137100 iteration: 3.9883916378
Loss after 137200 iteration: 1.29845023155
Loss after 137300 iteration: 1.21606910229
Loss after 137400 iteration: 2.62934470177
Loss after 137500 iteration: 1.34270966053
Loss after 137600 iteration: 0.939563989639
Loss after 137700 iteration: 1.38392055035
Loss after 137800 iteration: 1.02933037281
Loss after 137900 iteration: 1.26337945461
Loss after 138000 iteration: 4.28475809097
Loss after 138100 iteration: 1.9890152216
Loss after 138200 iteration: 0.93350225687
Loss after 138300 iteration: 1.39536774158
Loss after 1

PermissionDeniedError: /ModelWeights; Permission denied
	 [[Node: save/SaveV2 = SaveV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/SaveV2/tensor_names, save/SaveV2/shape_and_slices, beta1_power, beta2_power, dense/bias, dense/bias/Adam, dense/bias/Adam_1, dense/kernel, dense/kernel/Adam, dense/kernel/Adam_1, rnn/basic_lstm_cell/bias, rnn/basic_lstm_cell/bias/Adam, rnn/basic_lstm_cell/bias/Adam_1, rnn/basic_lstm_cell/kernel, rnn/basic_lstm_cell/kernel/Adam, rnn/basic_lstm_cell/kernel/Adam_1)]]

Caused by op u'save/SaveV2', defined at:
  File "/Users/maxime/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/maxime/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/maxime/anaconda2/li

# Tests

In [ ]:
mat = loadmat(train_addrs[0])
features = list(mat['structarr'][0,0])

In [ ]:
spike_raster_base = features[28].todense()

In [ ]:
spike_raster_base